# Subset Sum Problem
The [subset sum problem](https://en.wikipedia.org/wiki/Subset_sum_problem) is defined as, given a set of numbers, find a subset which adds up to another number.

## Implementation

For example let us have a set $S$ and a target $T$

$$
T = 7.0
$$

\begin{equation*}
S = 
\begin{bmatrix}
1.0 & 2.0 & 3.0 & 4.0 & 5.0 \\
\end{bmatrix}
\end{equation*}

Our goal is to find a mask $M$, such that, the dot product results in the target. Here is an example of a mask that adds up to our target.

\begin{equation*}
M = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 & 1.0 & 0.0 \\
\end{bmatrix}
\end{equation*}

We can verify that 

$$ T = M \cdot S $$


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, Input, layers

In [2]:
@tf.function
def compute_subset_sum(S, M):
    return tf.tensordot(S, M, 1)

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([0,0,1,1,0],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    T_ = compute_subset_sum(S, M)
    
print(T_)
print(tape.gradient(T_, S))
print(tape.gradient(T_, M))

tf.Tensor(7.0, shape=(), dtype=float32)
tf.Tensor([0. 0. 1. 1. 0.], shape=(5,), dtype=float32)
tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)


## Training

However, if we train as is, we find that $M$ is not a mask but it forms a linear combination with its inputs.

In [3]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        T_ = compute_subset_sum(S, M)
        loss = tf.nn.l2_loss(T_ - T)
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([1,1,1,1,1],dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        actual = compute_subset_sum(S, tf.round(M))
        tf.print(loss, M, T_, actual)

32 [0.994999826 0.994999826 0.994999826 0.994999826 0.994999826] 15 15
2.11328506 [0.601275 0.601274908 0.601274729 0.601274788 0.601274908] 9.05586243 15
0.0201957505 [0.479682386 0.479682267 0.479682088 0.479682148 0.479682267] 7.20097637 0
1.51871245e-05 [0.467018306 0.467018187 0.467018 0.467018068 0.467018187] 7.00551128 0
7.27595761e-12 [0.466667056 0.466666937 0.466666758 0.466666818 0.466666937] 7.00000381 0
5.57065505e-12 [0.466666609 0.46666649 0.466666311 0.466666371 0.46666649] 6.99999666 0
5.57065505e-12 [0.466666609 0.46666649 0.466666311 0.466666371 0.46666649] 6.99999666 0
5.57065505e-12 [0.466666609 0.46666649 0.466666311 0.466666371 0.46666649] 6.99999666 0
5.57065505e-12 [0.466666609 0.46666649 0.466666311 0.466666371 0.46666649] 6.99999666 0
5.57065505e-12 [0.466666609 0.46666649 0.466666311 0.466666371 0.46666649] 6.99999666 0


### Bistable loss
To force the values to be close to 0 and 1, we introduce the [Bistable Loss](notebooks/boolean-satisfiability.ipynb)

In [4]:
from library.loss import bistable_loss

On retraining we find that each element the mask is now closer to 0 or 1

In [5]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        T_ = compute_subset_sum(S, M)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([1,1,1,1,1],dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        actual = compute_subset_sum(S, tf.round(M))
        tf.print(loss, M, T_, actual)

32 [0.994999826 0.994999826 0.994999826 0.994999826 0.994999826] 15 15
5.2937994 [0.669517 0.625239313 0.614203155 0.609240353 0.606423259] 9.26705265 15
2.96882701 [0.707282722 0.499178141 0.474773556 0.466818601 0.463050485] 7.31969452 1
2.40181923 [0.987101078 0.454701811 0.422654927 0.420172453 0.420870364] 6.95283318 1
2.37065029 [1.01076198 0.424444467 0.392945588 0.407994449 0.419168055] 6.76682663 1
2.31917667 [1.01266503 0.372789234 0.362577498 0.415547 0.444725811] 6.73213863 1
2.06843424 [1.01433361 0.235568792 0.310641587 0.442599148 0.502211452] 6.69892883 6
1.52477646 [1.01065075 0.0557946712 0.212935656 0.499629229 0.609795094] 6.80628681 6
1.02521729 [0.999320805 0.00256707473 0.0553062148 0.550387084 0.734053254] 7.03983212 10
0.856600106 [0.989116728 -0.0184933655 -0.0223721061 0.554791 0.823360503] 7.2201438 10


### One hot softmax

To further make sure that the mask remains either 0 or 1, we increase the dimentionality of the $M$ and apply softmax along the vertical axis.

\begin{equation*}
M = 
\begin{bmatrix}
0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}
\end{equation*}

becomes

\begin{equation*}
M = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 1 \\
0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}
\end{equation*}

Therefore, $\bar{T}$ becomes

$$ M_s = softmax(M, axis=vertical) $$

$$ \bar{T} = \frac{(M_s[1] \cdot S) + ((1 - M_s[0]) \cdot S)}{2} $$

In [6]:
@tf.function
def compute_subset_sum_v2(S, M):
    M = tf.transpose(M)
    pos = tf.tensordot(S, M[1], 1)
    neg = tf.tensordot(S, 1 - M[0], 1)
    return (pos + neg) / 2

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable(tf.one_hot([0,0,1,1,0], 2),dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    M_s = tf.nn.softmax(M, axis=1)
    T_ = compute_subset_sum_v2(S, M_s)

tf.print(tf.transpose(M))
tf.print(T_)
tf.print(tape.gradient(T_, S))
tf.print(tape.gradient(T_, M))

[[1 1 0 0 1]
 [0 0 1 1 0]]
7.2689414
[0.268941402 0.268941402 0.731058598 0.731058598 0.268941402]
[[-0.196611926 0.196611941]
 [-0.393223852 0.393223882]
 [-0.589835823 0.589835823]
 [-0.786447763 0.786447704]
 [-0.983059645 0.983059704]]


In [7]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        M_s = tf.nn.softmax(M, axis=1)
        T_ = compute_subset_sum_v2(S, M_s)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M_s)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable(tf.one_hot([1,1,1,1,1], 2), dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        M_T = tf.transpose(M)
        M_T = tf.nn.softmax(M_T, axis=0)
        M_s = tf.nn.softmax(M, axis=1)
        actual = compute_subset_sum_v2(S, tf.round(M_s))
        tf.print(loss, M_T[1], T_, actual)
        
tf.print(M_T)

11.7297249 [0.729087949 0.729087889 0.729087889 0.729087889 0.729087889] 10.9658794 15
6.08404255 [0.849486172 0.522940934 0.5265553 0.527815342 0.528452575] 8.25115776 15
4.63299894 [0.936596215 0.315846235 0.405601978 0.432577103 0.444250286] 6.74399376 1
3.96075273 [0.961368561 0.159341335 0.334769219 0.4513897 0.492678] 6.5517168 1
3.10632539 [0.971199095 0.0969442874 0.204626277 0.5061602 0.608011901] 6.83957767 10
2.3400321 [0.975647 0.0659312457 0.110726014 0.537623584 0.725107] 7.21328068 10
1.93451571 [0.978034198 0.0484895259 0.0714 0.491285 0.801038265] 7.26118088 6
1.38491344 [0.980378 0.0397343 0.0554133914 0.338027656 0.86306566] 6.89832544 6
0.804450035 [0.983304262 0.0364712402 0.050469853 0.196737602 0.912564933] 6.55917883 6
0.570799768 [0.985896528 0.0352818146 0.0493095517 0.140285924 0.939776659] 6.46486235 6
[[0.0141034378 0.964718223 0.950690448 0.859714091 0.0602233037]
 [0.985896528 0.0352818146 0.0493095517 0.140285924 0.939776659]]


### Result

Bistable loss with one-hot performs much better.

## Neural network driven candidate generation

In the last few experiments, the initial candidate $M$ was arbitarily chosen. Here, we shall now delegate this *guesswork* to a neural network.

Let $G$ be our DNN. The specifics of its architecture is not relevant as this is a toy problem. $G$ takes in an set $S$ and a target $T$ and tries to guess the correct answer $\bar{M}$ in one shot.

$$ \bar{M} = G(S, T) $$

A benefit of having the `compute_subset_sum` function (lets call it $H$) end-to-end differentiable is that it can now be interfaced seamlessly with DNNs. 

We get

$$ \bar{T} = H(S, \bar{M}) $$

$$or$$

$$ \bar{T} = H(S, G(S, T)) $$

Now, we can use our bistable loss and $| \bar{T} - T | $ loss to train the system end-to-end.

In [8]:
class CandidateGeneratorBlock(layers.Layer):
    def __init__(self, set_length):
        super(CandidateGeneratorBlock, self).__init__()
        self.set_length = set_length
        
        self.dense1 = layers.Dense(10, kernel_initializer="he_normal", activation='relu')
        self.dense2 = layers.Dense(10, kernel_initializer="he_normal", activation='relu')
        self.dense3 = layers.Dense(set_length * 2)

    def call(self, x):
        h = x
        h = self.dense1(h)
        h = self.dense2(h)
        h = self.dense3(h)
        h = tf.reshape(h, (-1, self.set_length, 2))
        h = tf.nn.softmax(h, axis=-1)
        return h

In [9]:
batch_size = 1
set_length = 5
temp_generator = CandidateGeneratorBlock(set_length)
input_shape = (batch_size, set_length + 1)
output_shape = (batch_size, set_length, 2)
x = tf.random.normal(input_shape)
y = tf.math.round(tf.random.uniform(output_shape, minval=0, maxval=1))
result = temp_generator(x)
print(x.shape, result.shape, y.shape)
print(result)

(1, 6) (1, 5, 2) (1, 5, 2)
tf.Tensor(
[[[0.7139912  0.28600878]
  [0.47499245 0.5250076 ]
  [0.4091633  0.59083676]
  [0.27171898 0.7282811 ]
  [0.57239777 0.42760226]]], shape=(1, 5, 2), dtype=float32)


In [10]:
# batch_size = 10
# set_length = 5

# input_shape = (batch_size, set_length)
# output_shape = (batch_size, set_length, 2)
# x = tf.random.normal(input_shape)
# y = tf.math.round(tf.random.uniform(output_shape, minval=0, maxval=1))

# temp_generator = CandidateGeneratorBlock(set_length)
# a = Input(shape=(set_length))
# b = temp_generator(a)
# m = Model(inputs=a, outputs=b)
# m.compile(loss='mse', optimizer='adam')
# m.fit(x=x,y=y,epochs=100,batch_size=batch_size)

In [11]:
@tf.function
def compute_subset_sum_batch(S, M):
    M = tf.transpose(M,  [0, 2, 1])
    pos = tf.reduce_sum(S * M[:,1], axis=-1)
    neg = tf.reduce_sum(S * (1 - M[:,0]), axis=-1)
    return (pos + neg) / 2

dataset_size = 2
set_length = 5
h_S = np.random.randint(0, 10, (dataset_size, set_length))
h_M = np.random.randint(0, 2, (dataset_size, set_length))

S = tf.Variable(h_S, dtype=tf.float32)
M = tf.Variable(tf.one_hot(h_M, 2), dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    M_s = tf.nn.softmax(M, axis=-1)
    T_ = compute_subset_sum_batch(S, M_s)

tf.print(tf.transpose(M, [0, 2, 1]))
tf.print(T_)
tf.print(tape.gradient(T_, S))
tf.print(tape.gradient(T_, M))

[[[1 0 1 0 1]
  [0 1 0 1 0]]

 [[0 0 1 1 1]
  [1 1 0 0 0]]]
[16 4.30306244]
[[0.268941402 0.731058598 0.268941402 0.731058598 0.268941402]
 [0.731058598 0.731058598 0.268941402 0.268941402 0.268941402]]
[[[-1.76950729 1.76950753]
  [-1.37628365 1.37628353]
  [-0.786447704 0.786447763]
  [-1.76950753 1.76950729]
  [-0.589835823 0.589835823]]

 [[0 0]
  [0 0]
  [-1.37628353 1.37628365]
  [-0.786447704 0.786447763]
  [-0.983059645 0.983059704]]]


In [12]:
opt = tf.keras.optimizers.Adam()

set_length = 5
generator = CandidateGeneratorBlock(set_length)

@tf.function
def train_step(S, T, ST):
    with tf.GradientTape() as tape:
        M = generator(ST)
        T_ = compute_subset_sum_batch(S, M)
        TT = tf.transpose(T)
        loss = tf.reduce_mean(tf.math.squared_difference(TT, T_))
        bl = tf.reduce_sum(bistable_loss(M), axis=[1, 2]) * 10
        loss += tf.reduce_mean(bl)
    
    var_list = generator.trainable_variables
    grads = tape.gradient(loss, var_list)
    opt.apply_gradients(zip(grads, var_list))
    
    return loss, M, T_

In [13]:
dataset_size = 1000
set_length = 5
S = np.random.randint(0, 10, (dataset_size, set_length))
M = np.random.randint(0, 2, (dataset_size, set_length))
T = np.reshape(np.sum(S * M, axis=1), (dataset_size, 1))
ST = np.concatenate((S, T), axis=1)

S = np.float32(S)
T = np.float32(T)
ST = np.float32(ST)

In [14]:
train_step(S[:2], T[:2], ST[:2])

(<tf.Tensor: shape=(), dtype=float32, numpy=36.61646>,
 <tf.Tensor: shape=(2, 5, 2), dtype=float32, numpy=
 array([[[9.99797642e-01, 2.02336319e-04],
         [1.36642475e-05, 9.99986291e-01],
         [5.92471451e-07, 9.99999404e-01],
         [7.41534710e-01, 2.58465230e-01],
         [9.99999762e-01, 2.91080596e-07]],
 
        [[9.69633400e-01, 3.03665511e-02],
         [2.41654180e-02, 9.75834608e-01],
         [2.50900686e-01, 7.49099314e-01],
         [9.10448074e-01, 8.95519182e-02],
         [6.18431568e-01, 3.81568432e-01]]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([14.068474,  8.259684], dtype=float32)>)

In [15]:
s = tf.constant(S[0], dtype=tf.float32)
s = tf.expand_dims(s, 0)

for i in range(1000):
    loss, m, t_ = train_step(S, T, ST)
    if i % 100 == 0:
        m = m[0]
        t_ = t_[0]
        m_t = tf.transpose(m)
        r_m = tf.round(m)
        r_m = tf.expand_dims(r_m, 0)
        actual = compute_subset_sum_batch(s, r_m)[0]
        tf.print(loss, m_t[1], t_, actual, int(T[0,0]))
        
tf.print(m_t)

38.0997086 [0.000234175328 0.999985218 0.999999404 0.309308052 2.78606336e-07] 14.4753399 12 21
10.1025267 [0.181596816 0.999999046 0.999754846 0.947611749 7.96055222e-09] 20.3053169 20 21
6.48026657 [0.56387651 0.999999404 0.960552514 0.997461319 1.07887754e-09] 21.9196129 24 21
5.73290539 [0.756967843 0.999997735 0.944468737 0.999148846 2.01228881e-10] 22.5768032 24 21
5.04855633 [0.951817334 0.999989033 0.886824906 0.999789774 3.92590821e-11] 22.9001427 24 21
4.72130775 [0.991167903 0.999964237 0.827799082 0.9999578 9.12365409e-12] 22.5865822 24 21
4.49034357 [0.998867631 0.999913454 0.796109259 0.999991298 2.26091736e-12] 22.3639297 24 21
4.30261469 [0.999887347 0.999844432 0.768945515 0.999997735 6.6921631e-13] 22.1504745 24 21
4.15217876 [0.999986887 0.999717176 0.748130858 0.999998927 2.8707085e-13] 21.9838562 24 21
4.01752472 [0.99999845 0.999705493 0.727931 0.999999404 1.01167677e-13] 21.822258 24 21
[[1.5196631e-06 0.000294527214 0.272068977 5.36702942e-07 1]
 [0.99999845 0.9

In [16]:
nS = np.array([[1, 2, 3, 4, 5]], dtype=np.float32)
nT = np.array([[7]], dtype=np.float32)
nST = np.concatenate((nS, nT), axis=1)
nM = generator(nST)
np.round(nM)[0].T

array([[0., 0., 1., 0., 1.],
       [1., 1., 0., 1., 0.]], dtype=float32)

### Fine tuning the candidate

Once the neural network has guessed the initial candidate $\bar{M}$, we can now fine tune it using `compute_subset_sum` function. Assuming the neural network has learnt a good representation of the problem, the candidate should equal or at least be close to the solution. Therefore, we could expect it to converge faster and more reliably than manually guessing a candidate.

In [17]:
opt = tf.keras.optimizers.Adam()

@tf.function
def fine_tune_step(S, M, T):
    with tf.GradientTape() as tape:
        M_s = tf.nn.softmax(M, axis=1)
        T_ = compute_subset_sum_v2(S, M_s)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M_s)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable(nS,dtype=tf.float32)
M = tf.Variable(nM[0], dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = fine_tune_step(S, M, T)
    if i % 100 == 0:
        M_T = tf.transpose(M)
        M_T = tf.nn.softmax(M_T, axis=0)
        M_s = tf.nn.softmax(M, axis=1)
        actual = compute_subset_sum_v2(S, tf.round(M_s))
        tf.print(loss, M_T[1], T_[0], actual[0])
        
tf.print(M_T)

4.25990248 [0.723222 0.650265336 0.291595429 0.729313731 0.268548429] 7.15885067 7
3.55748272 [0.76129365 0.69552058 0.252263904 0.766952276 0.231619418] 7.13519096 7
2.89820313 [0.794480681 0.738415241 0.217733368 0.799760461 0.199965134] 7.12346792 7
2.3320303 [0.822267354 0.775921702 0.188515246 0.82707417 0.173519135] 7.11562 7
1.87315917 [0.845076859 0.807170928 0.164326832 0.84936887 0.151737064] 7.10862875 7
1.51269031 [0.863694 0.832624137 0.144478917 0.867491841 0.133874401] 7.10178614 7
1.2330004 [0.878932178 0.853232145 0.128189027 0.882288456 0.119186744] 7.09511614 7
1.01610804 [0.89149785 0.869977891 0.114743605 0.894473612 0.107026093] 7.08877087 7
0.846875072 [0.901959956 0.883702695 0.103550903 0.904613614 0.0968662798] 7.08286 7
0.713556945 [0.910761774 0.895073354 0.0941422805 0.913143694 0.0882939249] 7.07743168 7
[[0.0892381892 0.104926601 0.905857742 0.0868563354 0.91170609]
 [0.910761774 0.895073354 0.0941422805 0.913143694 0.0882939249]]
